<a target="_blank" href="https://colab.research.google.com/github/vanderbilt-data-science/ai_summer/blob/main/2_3-functions-tools-agents.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tools, Agents, and Assistants
> For Vanderbilt University AI Summer 2024<br>Prepared by Dr. Charreau Bell

_Code versions applicable: May 17, 2024_

## Learning Outcomes
* Participants will understand schemas and algorithms for usage in function calling and tool calling applications and articulate the behavior.
* Participants will be able to describe the behavior of agents and articulate fundamental advantages and disadvantages this approach.
* Participants will be able to leverage schemas, tool calling, function calling, agents, API assistants to enhance the capabilities and flexibility of their generative AI applications.

In [ ]:
! pip install langchain==0.1.20 langchain_openai sentence-transformers duckduckgo-search gradio
! pip install pypdf chromadb faiss-cpu

In [ ]:
import os

In [ ]:
# auth replicated here for reference just in case you choose to do something similar
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# enable tracing and set project name
os.environ['LANGCHAIN_TRACING_V2'] = "false"

# uncomment the following two lines before running the cell if you have a Langchain/Langsmith API Key
#os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
#os.environ['LANGCHAIN_TRACING_V2'] = "true"

# set langchain project
os.environ['LANGCHAIN_PROJECT'] = 'May17'

## Schemas
Resources:
* [Structured Output](https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/)

In [ ]:
from typing import Optional, List, Dict, Any
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# example interview information
interview_text = """My name is Jackson. I'm from Pittsburgh, Pennsylvania, and I graduated college in 2019.
I'm currently working as an administrative assistant at a law firm. I like to go hiking and play video games in my free time."""

In [ ]:
# specify the output schema
class PersonInformation(BaseModel):
    # add docstring

    # add fields
    graduation_year: int = Field(default=None, description="The year the person graduated.")
    job_title: str = Field(default='', description="The person's job title.")
    hobbies: List[str] = Field(default=[], description="The person's hobbies.")


In [ ]:
# use a model with function calling enabled
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_chatllm = #

In [ ]:
# create human prompt to send
text_prompt = f"Extract the person's name, home, graduation year, job title, and hobbies from the following interview text: \n\n{interview_text}"

# get structured output (hopefully)
structured_info = structured_chatllm.invoke(text_prompt)
structured_info

In [ ]:
structured_info.dict()

# Tools
## Function Calling
Resource: [Tool/Function Calling](https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/)

In [ ]:
from langchain_core.tools import tool

In [ ]:
# somewhat optional for this case
class MultiplySchema(BaseModel):
    """Multiply two integer values together. These values must be integers."""

    a: int = Field(..., description="First integer value")
    b: int = Field(..., description="Second integer value")

In [ ]:
# define tool

In [ ]:
# define available tools

tool_lookup = {'multiply': multiply}

In [ ]:
# create the model
chatllm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# binds the tools
tool_chatllm = 

In [ ]:
# use
tool_response = tool_chatllm.invoke("What is 6 times 5?")

# view response
tool_response

In [ ]:
# get responses

In [ ]:
# get function that is to be called
fn_to_call = #

# call function
tool_lookup[fn_to_call].invoke(# add invocation)

### Fitting into the function calling framework
Resource: [Langchain v2 Tool Calling](https://python.langchain.com/v0.2/docs/how_to/tool_calling/)

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

In [ ]:
query = "I have 6 books of 5 pages each. How many pages do I have in total?"
available_tools = {'multiply':multiply}

# get response from llm
llm_response = tool_chatllm.invoke(query)
llm_response

In [ ]:
# construct message history to complete the rest of the AI Message response
message_history = [HumanMessage(query), llm_response]

# make all tool calls
for tool_call in llm_response.tool_calls:
    # get the name of the called tool from the response
    called_tool_name = tool_call["name"]

    # get the tool object from the available tools
    selected_tool = available_tools[called_tool_name]

    # actually use the tool using the provided arguments
    tool_output = selected_tool.invoke(tool_call["args"])

    # append to the message history
    

# view the message history    
message_history

In [ ]:
# finish the call to the AI
final_answer = tool_chatllm.invoke(message_history)
final_answer

# Agents
Learn more about [Agents](https://python.langchain.com/docs/modules/agents/quick_start) in their Quickstart

**Chain Tool Invocation**
<figure>
<img src='https://python.langchain.com/v0.1/assets/images/tool_chain-3571e7fbc481d648aff93a2630f812ab.svg' height=300/>
    <figcaption>
        Source: Chain Tool Invocation, from <a href=https://python.langchain.com/v0.1/docs/use_cases/tool_use/>Calling tools with Chains</a>
    </figcaption>
</figure>

**Agent-based Tool Invocation**
<figure>
<img src='https://python.langchain.com/v0.1/assets/images/tool_agent-d25fafc271da3ee950ac1fba59cdf490.svg' height=400/>
    <figcaption>
        Source: Agent-based Tool Invocation, from <a href=https://python.langchain.com/v0.1/docs/use_cases/tool_use//>Calling tools with Agents</a>
    </figcaption>
</figure>


## Using Tools
Resource: [Q&A with RAG -> More -> Using Agents](https://python.langchain.com/v0.1/docs/use_cases/question_answering/conversational_retrieval_agents/)

### Choose Tools

In [ ]:
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


### Built in tools

In [ ]:
# create tool
search = #

# example
search.run("How many children does Barack Obama have?")

In [ ]:
# create tools
tools = #

# create llm with bound tools
search_llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
# create prompt
search_prompt = ChatPromptTemplate.from_messages([
    ('system', """You are a helpful assistant. You ALWAYS use the tools you have available instead of relying on internal information.
     You are brief and succinct in your responses. You use tools first to find information.
     You search the web to find answers about noteworthy figures or recent events."""),
    ('human', "{text}"),
    ('placeholder', "{agent_scratchpad}")
])

# create agent
agent = # create the agent

# Create an agent executor by passing in the agent and tools
agent_executor = # create the orchestrator of the overall excution


In [ ]:
# try it out
response = agent_executor.invoke({'text':"How many children does Barack Obama have?"})

### Custom tools

In [ ]:
# create tools
tools = # add more tools

# create agent
agent = # create tool calling agent

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

# try it out
response = agent_executor.invoke({'text':"""How many grand children would Barack Obama have if each of his children had 2 children?
                                  I heard he recently had a baby!"""})

## Retrievers as tools

In [ ]:
from bs4 import SoupStrainer
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Creating custom retriever tools
from langchain.tools.retriever import create_retriever_tool

In [ ]:
# use website data
sw_website = 'https://simple.wikipedia.org/wiki/Star_Wars_Episode_IV:_A_New_Hope'
webloader = WebBaseLoader(sw_website,
                       bs_kwargs = {'parse_only':SoupStrainer('div', id='bodyContent')})
web_chunks = webloader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True))
print('Number of chunks generated: ', len(web_chunks))

# create embeddings
embeddings_fn = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") #, model_kwargs={"device":'mps'})
hf_db = FAISS.from_documents(web_chunks, embeddings_fn)
hf_retriever = hf_db.as_retriever(search_kwargs={"k":5})

In [ ]:
tool_name = "Information Retriever for the Star Wars Movie: A New Hope"
description = """You MUST use this tool whenever any information is requested about the Star Wars movie: A New Hope.
    Make sure to rephrase the query so that it is not a question, but a statement to be compared with
    text about the movie."""

# based off of the Guide we used
retriever_tool = create_retriever_tool(
    hf_retriever,
    # add name
    # add description
)

In [ ]:
# define tools
tools = # setup tool

In [ ]:
# create prompt
react_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(template='Answer the following questions as best you can or reply with the most meaningful ' +
                                             'response possible. You have access to the following tools:\n\n{tools}\n\n although you can also reply conversationally when appropriate. '
                                             'Use the following format to reason out your response:\n\n' +
                                             'Question: the input question you must answer or statement to which you should reply\n' +
                                             'Thought: you should always think about what to do\n' +
                                             'Action: the action to take, should be one of [{tool_names}]\n'+
                                             'Action Input: the input to the action, making sure that the inputs are in the valid format for the action\n'+
                                             'Observation: the result of the action\n... '+
                                             '(this Thought/Action/Action Input/Observation can repeat up to 3 times)\n'+
                                             'Thought: I now know the final answer\n'+
                                             'Final Answer: the final answer to the original input question or appropriate response is\n\n'+
                                             'Begin!\n\nQuestion or message: {input}\nThought:{agent_scratchpad}'),
  ]
)

In [ ]:
# Create our own llm and agent
agent = # create another type of agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, run_intermediate_steps=True)

In [ ]:
# Execute the agent
agent_executor.invoke(
    {"input": # add a question}
)

In [ ]:
query = """What did Obi-Wan do for Luke Skywalker in a New Hope?
How did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies."""

# Execute the agent
agent_executor.invoke(
    {"input": #add a query}
)

## Adding the conversational component

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# start with empty chat history. We can pass this as a parameter in the invoke


# Create the agent with chat history
agent_with_chat_history = RunnableWithMessageHistory(
            # add required agent
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    handle_parsing_errors = "Check your output and make sure it conforms, use the Action/Action Input syntax"
)

In [ ]:
# Start chatting away
query_1 = """What did Obi-Wan do for Luke Skywalker in a New Hope?
How did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies."""

session_id_1 = "session_1"

# Execute the agent
agent_with_chat_history.invoke({"input":# add query}, config={"configurable": {"session_id": session_id_1}})

In [ ]:
# Start chatting away
query_2 = """Elaborate on your previous answer, and provide details that reference the object that Obi-Wan gave to Luke Skywalker in A New Hope."""

# Execute the agent
agent_with_chat_history.invoke({"input": # follow up conversation}, config={"configurable": {"session_id": session_id_1}})

# Bringing it all together
With a user history, a tool, and a retriever, you can now create a conversational agent through a quick UI that can answer questions and provide information.

In [ ]:
import gradio as gr

In [ ]:
message_history.clear()
session_id_chat = 'session_id_chat'

In [ ]:
# create response
def ai_response(user_message, chat_info):
    agent_message = {'input':user_message}
    response = agent_with_chat_history.invoke(agent_message, config={"configurable": {"session_id": session_id_chat}})
    conversation = message_history.messages
    formatted_messages = [(conversation[ind].content, conversation[ind+1].content)
                           for ind in range(0, len(conversation), 2)]
    return '', formatted_messages

# create gradio blocks UI
with gr.Blocks() as demo :
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(ai_response, [msg, chatbot], [msg, chatbot])

demo.launch()

# OpenAI Assistants - Agents! (?)
Resource: [Assistants API](https://platform.openai.com/docs/assistants/overview)

Resource: [Platform Documentation - time for the playground!](https://platform.openai.com/playground/assistants)

## Conversation

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# create assistant with code interpreter
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter", "type": "file_search"}],
    model="gpt-4o",
)

EASY message history!

In [ ]:
# create a thread of message history
thread = client.beta.threads.create()
thread

In [ ]:
# add a message to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

In [ ]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="The user prefers to have concepts explained to them like they're 5. Then, describe it at an appropriate college level."
)

In [ ]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print('Finished running.')
else:
  print(run.status)

messages.model_dump()

In [ ]:
# get message information
print(len(messages.data))

In [ ]:
# view the ai response
print(messages.data[0].content[0].text.value)

In [ ]:
# view all messages
for ind, message in enumerate(messages.data):
    print(f'***** Message {ind}:\n{message.content[0].text.value}\n *****')

In [ ]:
# view entire run information
run.model_dump()

## Tools: Code Interpreter
Resource: [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter)

In [ ]:
! curl -o palmer_penguins.csv https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv

In [ ]:
# Upload the penguins file to be used directly with a message thread with code interpreter
palmer_penguins_data = client.files.create(
    file=open("palmer_penguins.csv", "rb"),
    purpose='assistants'
)

In [ ]:
# modify thread to include the penguins file
ci_thread = client.beta.threads.create(
    tool_resources={"code_interpreter": {"file_ids": [palmer_penguins_data.id]}}
)

In [ ]:
# create new message to call palmer penguins
message = client.beta.threads.messages.create(
    thread_id=ci_thread.id,
    role="user",
    content="I need to analyze the Palmer Penguins data I uploaded. Create a histogram reflecting the counts of the different species of penguins.",
    attachments=[
        {
            "file_id":palmer_penguins_data.id,
            "tools":[{"type":"code_interpreter"}]
        }
    ]
)

In [ ]:
# create run and force use of code interpreter
run = client.beta.threads.runs.create_and_poll(
  thread_id=ci_thread.id,
  assistant_id=assistant.id,
  tools=[{"type":"code_interpreter"}],
  tool_choice = {"type":"code_interpreter"}
)

In [ ]:
# check completion and view output
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=ci_thread.id
  )
  print('Finished running.')
else:
  print(run.status)

messages.model_dump()
#run.model_dump()

In [ ]:
len(messages.data)

: 

In [ ]:
# retrieve a single image file
#generated_image = messages.data[0].content[0].image_file.file_id
#print(generated_image)

# download using file api
#image_data = client.files.content(generated_image)
#image_data_bytes = image_data.read()

## Retrieval and Vector Stores
Resource: [File Search](https://platform.openai.com/docs/assistants/tools/file-search/file-search-beta)

In [ ]:
! curl -o declaration_independence.pdf https://www.uscis.gov/sites/default/files/document/guides/M-654.pdf

In [ ]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="declaration of independence")
 
# Ready the files for upload to OpenAI
file_paths = ["declaration_independence.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

In [ ]:
# create assistant with code interpreter
history_assistant = client.beta.assistants.create(
    name="Prominent H. Figure",
    instructions="You are a prominent historical figure from the late 1770s. You're an opinionated dynamic oratorical speaker, but you make your point succinctly!",
    tools=[{"type": "file_search"}],
    model="gpt-3.5-turbo",
)

In [ ]:
history_assistant = client.beta.assistants.update(
  assistant_id=history_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
# create thread for history
history_thread = client.beta.threads.create()
history_thread

In [ ]:
# create and send message
message = client.beta.threads.messages.create(
  thread_id=history_thread.id,
  role="user",
  content="What reasons do the authors cite for declaring independence from Great Britain in the Declaration of Independence?"
)
message

In [ ]:
# create run and force use of code interpreter
run = client.beta.threads.runs.create_and_poll(
  thread_id=history_thread.id,
  assistant_id=history_assistant.id,
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=history_thread.id
  )
  print('Finished running.')
else:
  print(run.status)

In [ ]:
#view result
messages.model_dump()

# Congratulations!! 
You have now learned how to use tools, agents, and assistants to enhance the capabilities of your generative AI applications. You have experience using the OpenAI Completions and Assistants API, are able to execute RAG with conversation history, and are able to port this knowledge to also using agents and tools.

Next week, we'll start model training - congratulations again!